# EEP 153 Project 2 Subsistence Cost Diet: Atwater

In [1]:
# API key for Gov;
apikey = "bwFohFv0W79JagEjhjfy121CHf29UEljz0OYel1N"

## Dietary Reference Intakes

In [2]:
# read in dietary requirements (max and min)
import pandas as pd
dri_max = pd.read_csv("Dietary Requirements Max.csv").set_index('Nutrition')

dri_min = pd.read_csv("Dietary Requirements Min.csv").set_index('Nutrition')

# convert kcal to kJ
temp = dri_max.loc['Energy']
temp.iloc[1:] = temp.iloc[1:] * 4.184
dri_max.loc['Energy'] = temp

temp = dri_min.loc['Energy']
temp.iloc[1:] = temp.iloc[1:] * 4.184
dri_min.loc['Energy'] = temp

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
import re
# function for age-sex specific DRI
def dri(age,sex,dietmin,dietmax):
    if age <= 3:
        index = 2

    else:
        for i, j in enumerate(list(dietmax)[3:]):
            if j[0] == sex:
                interval = re.findall(r'\d+', j)
                if len(interval) == 1:
                    if age >= int(interval[0]):
                        index = i + 3
                        break
                else:
                    if age >= int(interval[0]) and age <= int(interval[1]):
                        index = i + 3
                        break

    df = pd.DataFrame({'Nutrition': dietmin.iloc[:, 0],
                       'Max/Min': 'Minimum',
                       'Age & Sex': list(dietmin)[index],
                      'Intake': dietmin.iloc[:, index]})   
    df_max = pd.DataFrame({'Nutrition': dietmax.iloc[:, 0],
                       'Max/Min': 'Maximum',
                       'Age & Sex': list(dietmax)[index],
                      'Intake': dietmax.iloc[:, index]})   

    df = pd.concat([df, df_max], axis=0)
    
    return df
    

In [4]:
# Dietary reference intakes for 10 years old female
dri(10, 'F', dri_min, dri_max)

,Nutrition,Max/Min,Age & Sex,Intake
Nutrition,,,,
Energy,---,Minimum,F 9-13,6694.4
Protein,RDA,Minimum,F 9-13,34.0
"Fiber, total dietary",---,Minimum,F 9-13,22.4
"Folate, DFE",RDA,Minimum,F 9-13,300.0
"Calcium, Ca",RDA,Minimum,F 9-13,1300.0
"Carbohydrate, by difference",RDA,Minimum,F 9-13,130.0
"Iron, Fe",RDA,Minimum,F 9-13,8.0
"Magnesium, Mg",RDA,Minimum,F 9-13,240.0
Niacin,RDA,Minimum,F 9-13,12.0


## Data on prices for different foods

In [5]:
# get food list from five restaurants
food_list_total = pd.read_csv("FoodList1.csv")
food_list_total = food_list_total.astype({"FDC": str})
food_list_total

,Restaurant,Dish,Ingredients,Dish_Price,Quantity,Unit,FDC,Calorie/100g,Ingredient_Price/100 gm
0,Chipotle,Chicken burrito,chicken,8.85,132.0000,gram,331960,237.0,1.320
1,Chipotle,Chicken burrito,white rice,8.85,120.0000,gram,790214,359.0,0.274
2,Chipotle,Chicken burrito,black beans,8.85,60.0000,gram,747444,180.0,0.440
3,Chipotle,Chicken burrito,lettuce,8.85,60.0000,gram,2346389,21.0,1.110
4,Chipotle,Chicken burrito,salsa,8.85,40.0000,gram,746777,34.0,0.590
...,...,...,...,...,...,...,...,...,...
315,Poke Bar,Wazzup Poke Bowl,cucumber,15.95,28.3500,gram,168409,15.0,0.880
316,Poke Bar,Wazzup Poke Bowl,green onion,15.95,7.0875,gram,170006,27.0,0.440
317,Poke Bar,Wazzup Poke Bowl,ponzu,15.95,35.4375,gram,2451144,33.0,1.900
318,Poke Bar,Wazzup Poke Bowl,wasabi,15.95,452.1825,gram,171831,292.0,8.780


In [6]:
# split food_list by restaurant
grouped = food_list_total.groupby(food_list_total.Restaurant)
restaurants = ['Thai Basil', 'Ttoust', 'IB', 'Poke Bar']

food_list_res = []

for r in restaurants: 
    food_list_res.append(grouped.get_group(r))


In [7]:
import fooddatacentral as fdc
import warnings
from collections import defaultdict

# get nutritional information for ingredients
ing_res = []
for i in range(4):
    L = []
    D = {}
    items = []
    count = 0
    food_list = food_list_res[i]
    for food in food_list.Ingredients.tolist():
        try:
            FDC = food_list.iloc[count,:].FDC
            count+=1
            temp = fdc.nutrients(apikey,FDC)
            key = temp.Units
            # convert units if necessary
            if 'Energy' in key.index:
                if key['Energy'] != 'kJ':
                    temp.Quantity['Energy'] = temp.Quantity['Energy']*4.184

            L.append(temp.Quantity)
            D[food] = temp.Quantity
            items.append(food)
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))     

    FoodNutrients_Ing = pd.DataFrame(L,dtype=float)
    FoodNutrients_Ing.index = items
    FoodNutrients_Ing = FoodNutrients_Ing.fillna(0)
    FoodNutrients_Ing = FoodNutrients_Ing.transpose() 

    #FoodNutrients_Ing

    FoodNutrients_Ing_d = pd.DataFrame(D,dtype=float)
    FoodNutrients_Ing_d = FoodNutrients_Ing_d.fillna(0)
    ing_res.append(FoodNutrients_Ing_d)

/tmp/ipykernel_198/162337130.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.Quantity['Energy'] = temp.Quantity['Energy']*4.184


In [8]:
ing_res[0]

,prawns,chicken,egg,Pineapple,white onion,raisins,cashew nuts,peas,carrots,bell pepper,...,bean sprouts,Thai chili,peanuts,tofu,beef,pork,imitation crab,mushrooms,catfish,red curry
"Ergosta-5,7-dienol",0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,...,0.000,0.0,0.000,0.0,0.0,0.00,0.0,3.607,0.00,0.0
"Ergosta-7,22-dienol",0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,...,0.000,0.0,0.000,0.0,0.0,0.00,0.0,1.507,0.00,0.0
25-hydroxycholecalciferol,0.0,0.0,0.560,0.0,0.0,0.0,0.000,0.00,0.0,0.0,...,0.000,0.0,0.000,0.0,0.0,0.00,0.0,0.000,0.00,0.0
Alanine,0.0,0.0,0.667,0.0,0.0,0.0,0.837,0.24,0.0,0.0,...,0.174,0.0,1.001,0.0,0.0,0.53,0.0,0.000,0.86,0.0
"Alcohol, ethyl",0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,...,0.000,0.0,0.000,0.0,0.0,0.00,0.0,0.000,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cis-Lutein/Zeaxanthin,0.0,0.0,57.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,...,0.000,0.0,0.000,0.0,0.0,0.00,0.0,0.000,0.00,0.0
cis-Lycopene,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,...,0.000,0.0,0.000,0.0,0.0,0.00,0.0,0.000,0.00,0.0
cis-beta-Carotene,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,...,0.000,0.0,0.000,0.0,0.0,0.00,0.0,0.000,0.00,0.0
trans-Lycopene,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,...,0.000,0.0,0.000,0.0,0.0,0.00,0.0,0.000,0.00,0.0


In [9]:
# Convert food quantities to FDC units
food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))

# Now may want to filter df by time or place--need to get a unique set of food names.
food_list['FDC Price'] = food_list['Dish_Price']/food_list.groupby('Dish',sort=False)['FDC Quantity'].sum()

food_list.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Dish_Prices = food_list.groupby('Dish',sort=False)['Dish_Price'].min()/food_list.groupby('Dish',sort=False)['FDC Quantity'].sum()

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/tmp/ipykernel_198/2289315912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))
/tmp/ipykernel_198/2289315912.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_list['FDC Price'] = food_list['Dish_Price']/food_

## Table of nutritional information for dishes

In [10]:
# construct table of nutritional information for dishes

dishes = food_list.groupby('Dish',sort=False)['Ingredients'].count()

FoodNutrients = pd.DataFrame(columns=dishes.keys().tolist(),
                  index=FoodNutrients_Ing.index, )
start = 0
end = 0
for i, column in enumerate(FoodNutrients): 

    FoodNutrients[column] = FoodNutrients_Ing.iloc[:, start:start+dishes[i]].sum(axis=1)
    start = start+dishes[i]
FoodNutrients

,Original Salmon Poke Bowl,Firecracker Poke Bowl,Sunset House Poke Bowl,The O.G. Poke Bowl,Goodie Mob Poke Bowl,Wazzup Poke Bowl
Proximates,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
Water,349.970000,349.370000,424.540000,336.83000,444.350000,448.360000
Energy,14278.841536,50999.296902,42197.319302,44274.83239,5629.147776,5466.693248
Protein,42.170000,30.610000,56.240000,44.55000,41.900000,33.200000
Total lipid (fat),81.880000,120.000000,71.250000,63.89000,13.850000,46.890000
...,...,...,...,...,...,...
"Fatty acids, total trans-polyenoic",0.000000,0.000000,0.000000,0.00000,0.000000,0.023000
"Fluoride, F",0.000000,1.300000,1.300000,0.00000,1.300000,1.300000
Phytosterols,0.000000,14.000000,14.000000,0.00000,14.000000,14.000000
"Sugars, added",0.000000,13.300000,13.300000,13.30000,0.000000,0.000000


In [11]:
Dish_Prices

Dish
Original Salmon Poke Bowl    3.2149155958679763 / hectogram
Firecracker Poke Bowl         6.082272748939414 / hectogram
Sunset House Poke Bowl       2.6790963298899806 / hectogram
The O.G. Poke Bowl           1.7337757450751807 / hectogram
Goodie Mob Poke Bowl         1.7472367368847699 / hectogram
Wazzup Poke Bowl             1.6819438842358623 / hectogram
dtype: object

In [12]:
# Convert food quantities to FDC units
ing_prices = []
for r in food_list_res:
    food_list = r
    food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))

    # Now may want to filter df by time or place--need to get a unique set of food names.
    food_list['FDC Price'] = food_list['Ingredient_Price/100 gm']

    food_list.dropna(how='any') # Drop food with any missing data

    # To use minimum price observed
    Ingredient_Prices = food_list.groupby('Ingredients',sort=False)['FDC Price'].min()
    ing_prices.append(Ingredient_Prices)

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/tmp/ipykernel_198/2842241641.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))
/tmp/ipykernel_198/2842241641.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_list['FDC Price'] = food_list['Ingredient_Price/1

In [13]:
# Ingredient price for Poke bar
ing_prices[3]

Ingredients
salmon            2.20
white rice        0.28
sweet onion       0.44
green onion       0.44
kale              0.88
shoyu             1.69
spicy mayo        1.38
ahi tuna          2.36
cucumber          0.88
sweet chili       1.05
house dressing    3.17
edamame           0.56
ponzu             1.90
tofu              0.74
wasabi            8.78
mayo              0.56
Name: FDC Price, dtype: float64

## Solution

In [14]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)
    #print(Aall)
    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    #print(dietmin)
    #print(Amin)
    Amin = Amin.reindex(dietmin.index,axis=0)
    #print(dietmin.index)
    #print(Amin)
    idx = Amin.index.to_frame()
    #print(Amin)
    idx['type'] = 'min'
    #print(Amin)
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    

    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
    #print(p)
    #print(A)
    #print(b)
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point', options={'presolve': True,
                                                            'cholesky':False, 
                                                            'sym_pos':False,
                                                            'lstsq':True})

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

### Dish-based Solution

In [15]:
#dish-based result
group = 'M 19-30'
tol = 1e-5

result = solve_subsistence_problem(FoodNutrients,Dish_Prices,dri_min[group],dri_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())


Cost of diet for M 19-30 is $14.37 per day.


Diet (in 100s of grams or milliliters):
Series([], dtype: float64)


With the following nutritional outcomes of interest:
                                Outcome  Recommendation
Nutrition                                              
Energy                              NaN         10041.6
Protein                             NaN            56.0
Fiber, total dietary                NaN            33.6
Folate, DFE                         NaN           400.0
Calcium, Ca                         NaN          1000.0
Carbohydrate, by difference         NaN           130.0
Iron, Fe                            NaN             8.0
Magnesium, Mg                       NaN           400.0
Niacin                              NaN            16.0
Phosphorus, P                       NaN           700.0
Potassium, K                        NaN          4700.0
Riboflavin                          NaN             1.3
Thiamin                             NaN         

/tmp/ipykernel_198/2046845037.py:91: UserWarning: The algorithm terminated successfully and determined that the problem is infeasible.
  warnings.warn(result.message)


### Ingredient-based solutions

In [16]:
# ingredient-based results for each of 4 restaurants
groups = dri_min.columns[1:]

cost_tbl = pd.DataFrame(columns = groups, index = restaurants)
diet_tbl = pd.DataFrame(columns = groups, index = restaurants)
nurt_tbl = pd.DataFrame(columns = groups, index = restaurants)
excess_tbl = pd.DataFrame(columns = groups, index = restaurants)

for group in groups:
    tol = 1e-6
    for i in range(4):
        result = solve_subsistence_problem(ing_res[i],ing_prices[i],dri_min[group],dri_max[group],tol=tol)
        
        #results_tbl.loc[restaurants[i], group] = result
        
        print("Result for restaurant: " + restaurants[i])
        print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))
        
        cost_tbl.loc[restaurants[i], group] = result.fun
        
        # Put back into nice series
        diet = result.diet

        print("\nDiet (in 100s of grams or milliliters):")
        print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
        print()
        
        diet_tbl.loc[restaurants[i], group] = list(diet[diet >= tol])

        tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
        print("\nWith the following nutritional outcomes of interest:")
        print(tab)
        print()
        
        nurt_tbl.loc[restaurants[i], group] = list(np.abs(result.A).dot(diet))

        print("\nConstraining nutrients are:")
        excess = tab.diff(axis=1).iloc[:,1]
        print(excess.loc[np.abs(excess) < tol*100].index.tolist())
        
        excess_tbl.loc[restaurants[i], group] = list(excess.loc[np.abs(excess) < tol*100].index.tolist())

/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp

Result for restaurant: Thai Basil
Cost of diet for C 1-3 is $4.05 per day.


Diet (in 100s of grams or milliliters):
egg         1.653595
carrots     1.385275
rice        0.709294
broccoli    0.294118
peanuts     3.263896
tofu        1.812969
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          10459.999995          4184.0
Protein                           129.501998            13.0
Fiber, total dietary               34.722741            14.0
Folate, DFE                       780.071050           150.0
Calcium, Ca                       700.000000           700.0
Carbohydrate, by difference       130.000000           130.0
Iron, Fe                           14.715686             7.0
Magnesium, Mg                     616.643101            80.0
Niacin                             43.412963             6.0
Phosphorus, P   

/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")


Result for restaurant: Poke Bar
Cost of diet for F 4-8 is $8.56 per day.


Diet (in 100s of grams or milliliters):
salmon        0.371517
white rice    0.923280
kale          6.489605
edamame       3.167269
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          10459.999993          5020.8
Protein                            68.453133            19.0
Fiber, total dietary               44.659597            16.8
Folate, DFE                      1321.547001           200.0
Calcium, Ca                      1854.139526          1000.0
Carbohydrate, by difference       130.000000           130.0
Iron, Fe                           17.800823            10.0
Magnesium, Mg                     452.873396           130.0
Niacin                             16.077025             8.0
Phosphorus, P                    1056.005900        

/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")


...

/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")


Result for restaurant: Thai Basil
Cost of diet for M 51+ is $6.69 per day.


Diet (in 100s of grams or milliliters):
egg         4.947712
carrots     3.846579
rice        0.179987
broccoli    1.176471
peanuts     3.770187
tofu        1.708201
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          12970.399999          8368.0
Protein                           182.869415            56.0
Fiber, total dietary               48.402653            28.0
Folate, DFE                       901.074603           400.0
Calcium, Ca                      1000.000000          1000.0
Carbohydrate, by difference       130.000000           130.0
Iron, Fe                           22.193402             8.0
Magnesium, Mg                     777.823325           420.0
Niacin                             53.041098            16.0
Phosphorus, P   

/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")


Result for restaurant: IB
Cost of diet for M 51+ is $7.20 per day.


Diet (in 100s of grams or milliliters):
bun                0.416821
ketchup            0.221608
lettuce            1.015800
cheddar cheese     0.757351
sandwich bun       0.416821
avocado           11.906505
cheese             0.757351
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          12045.958131          8368.0
Protein                            75.227442            56.0
Fiber, total dietary               83.184902            28.0
Folate, DFE                      1192.022992           400.0
Calcium, Ca                      1396.454973          1000.0
Carbohydrate, by difference       130.000000           130.0
Iron, Fe                           10.424396             8.0
Magnesium, Mg                     420.000000           420.0
Niacin         

/tmp/ipykernel_198/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")


In [17]:
cost_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Thai Basil,4.052806,5.681346,5.681348,7.366037,7.347198,7.559409,7.889342,6.257664,6.693508,6.257664,6.693506,7.163111,6.693506
Ttoust,3.254694,4.524609,4.524609,5.881514,5.88149,6.141627,6.160926,5.353081,5.41441,5.353081,5.41441,5.832565,5.41441
IB,5.046551,5.973081,5.97308,6.050696,6.050574,9.070692,7.207749,10.594395,7.080005,10.594395,7.198854,6.485626,7.198853
Poke Bar,6.377665,8.560379,8.560379,10.494406,10.194235,12.590013,15.451584,12.504119,15.451584,12.504119,15.451586,12.504119,15.451584


In [18]:
diet_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Thai Basil,"[1.6535947706889553, 1.3852752287717585, 0.709...","[2.1982570758309756, 4.11231494774835, 0.34127...","[2.198256851418826, 1.78151173378974e-06, 4.11...","[3.3071895265853226, 5.56034958804642, 0.07673...","[3.3071895423056197, 4.722821936295839, 0.1153...","[3.8562091513207064, 4.790331164253986, 0.0718...","[4.925740434841197, 0.20758255836004372, 4.847...","[3.849673202078504, 3.9686114560475874, 0.1365...","[4.947712189157788, 1.9233930755712387e-06, 3....","[3.849673201126317, 3.968611957532809, 0.13658...","[4.94771241775595, 3.8465795947305943, 0.17998...","[3.8496732024200333, 4.427298237925396, 0.0987...","[4.947712418235566, 3.8465794220684857, 0.1799..."
Ttoust,"[0.882352925603457, 1.5314822731509792, 0.2748...","[1.1764705868476217, 2.551020416219487, 0.4147...","[1.1764705880371733, 2.5510204034365738, 0.414...","[1.7647058822237875, 4.591836734301037, 0.4271...","[1.7647058820567403, 4.591836733460813, 0.4222...","[2.3529411759875267, 6.632653048006388, 0.1522...","[2.352941175766801, 0.038373509910166095, 7.42...","[0.899093957300926, 0.2563041354425244, 6.1623...","[0.7481425468840616, 0.515229448839125, 6.1869...","[0.8990940417231688, 0.2563041491157663, 6.162...","[0.7481426971027444, 0.5152294766063318, 6.186...","[2.3529411763500763, 0.05379011503111703, 7.34...","[0.7481427032552965, 0.5152294773758332, 6.186..."
IB,"[0.12140266060050481, 0.8973065484928846, 2.49...","[0.7381884201213375, 0.36265548456572066, 0.60...","[0.7381884087079148, 0.3626555521447199, 0.607...","[0.1818589658506186, 0.9773167656716606, 0.555...","[0.18185436510852976, 0.975725026242236, 0.551...","[1.2233763054815683, 1.57576435130077e-06, 2.1...","[0.5000167883018362, 0.025090358481825042, 1.0...","[1.5156130817054203, 3.8785864811010535, 0.173...","[0.5156678917103827, 0.31452057376256354, 1.20...","[1.5156130812067994, 3.8785864897315094, 0.173...","[0.41682131618836893, 0.22160690879118547, 1.0...","[0.6185488197692462, 0.7183637701925146, 0.656...","[0.41682140001715734, 0.2216078528273798, 1.01..."
Poke Bar,"[0.2786377700974017, 1.04206165823365, 4.13955...","[0.3715170277047409, 0.923279850987226, 6.4896...","[0.37151702781763934, 0.9232798518582935, 6.48...","[0.6315231559302386, 0.8115626100037986, 1.611...","[0.593710832494077, 0.7905590873998726, 1.4035...","[1.8430382611738536, 0.8342380454859687, 7.086...","[0.6937842576074815, 0.48377537465033155, 12.9...","[1.8802723382680828, 0.8456054989862727, 6.898...","[0.6937842973729921, 0.4837754199649512, 12.93...","[1.8802723187799568, 0.8456055203133529, 6.898...","[0.6937844327917748, 0.4837758079467785, 2.984...","[1.880272337983991, 0.8456054990230503, 6.8981...","[0.693784262838222, 0.4837754128303332, 12.933..."


In [19]:
nurt_tbl.iloc[1,1]

[10459.998923079153,
 84.68608960743502,
 86.21945718594837,
 573.4175691360044,
 1000.0000000445157,
 449.84459176570596,
 38.40419387870724,
 1052.5833199392803,
 33.818038202406456,
 2474.4401938047595,
 3800.0000001348694,
 2.2766723886068054,
 2.365975976282556,
 1548.4994043794954,
 1.2000000004507179,
 2.115576431538548,
 25.00000030530556,
 8.452376544520046,
 1270.0910082076123,
 23.60424297294048,
 352.1202874184242,
 10459.998923079153]

In [20]:
excess_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Thai Basil,"[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Vit...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot..."
Ttoust,"[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Niacin, Potassium, K, Vitamin B-...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita...","[Energy, Calcium, Ca, Potassium, K, Vitamin B-...","[Calcium, Ca, Potassium, K, Vitamin B-12, Vita..."
IB,"[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Iron, Fe, Sodium, Na]","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Iron, Fe, Sodium, Na, Energy]","[Carbohydrate, by difference, Magnesium, Mg, T...","[Calcium, Ca, Iron, Fe, Sodium, Na, Energy]","[Carbohydrate, by difference, Magnesium, Mg, T...","[Calcium, Ca, Carbohydrate, by difference, Thi...","[Carbohydrate, by difference, Magnesium, Mg, T..."
Poke Bar,"[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Phosphorus, P, P...","[Carbohydrate, by difference, Vitamin E (alpha...","[Carbohydrate, by difference, Vitamin E (alpha...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Vitamin E (alpha...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Potassium, K, Vi...","[Carbohydrate, by difference, Vitamin B-12, Vi..."


## Sensitivity of solution

### Effects of Price Changes on Subsistence Diet Cost

In [21]:
# examine effects of price change on subsistence diet cost for T-toust, the cheapest option
# for men and women aged 19-30
import cufflinks as cf
cf.go_offline()

group_pc = ['F 19-30', 'M 19-30']

for group in group_pc: 
    scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

    cost0 = solve_subsistence_problem(ing_res[1],ing_prices[1],dri_min[group],dri_max[group],tol=tol).fun

    Price_response={}
    for s in scale:
        cost = {}
        for i,p in enumerate(ing_prices[1]):
            my_p = ing_prices[1].copy()
            my_p[i] = p*s
            result = solve_subsistence_problem(ing_res[1],my_p,dri_min[group],dri_max[group],tol=tol)
            cost[ing_prices[1].index[i]] = np.log(result.fun/cost0)
        Price_response[np.log(s)] = cost

    Price_response = pd.DataFrame(Price_response).T
    Price_response.iplot(xTitle='change in log price',yTitle='change in log cost', title ='Effects of Price Changes on Subsistence Diet Cost for '+ group)

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.



/tmp/ipykernel_198/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



### Effects of Price Changes on Subsistence Diet Composition

In [22]:
# examine effects of price change on subsistence diet cost for T-toust, the cheapest option
# for men and women aged 19-30
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'Egg'

group_pc = ['F 19-30', 'M 19-30']

for group in group_pc: 

    scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

    cost0 = solve_subsistence_problem(ing_res[1],ing_prices[1],dri_min[group],dri_max[group],tol=tol).fun

    my_p = ing_prices[1].copy()
    diet = {}
    for s in scale:
        my_p[ReferenceGood] = ing_prices[1][ReferenceGood]*s
        result = solve_subsistence_problem(ing_res[1],my_p,dri_min[group],dri_max[group],tol=tol)
        diet[my_p[ReferenceGood]] = result.diet

    Diet_response = pd.DataFrame(diet).T
    Diet_response.index.name = '%s Price' % ReferenceGood

    Diet_response.reset_index(inplace=True)

    # Get rid of units for index (cufflinks chokes)
    # Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

    Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

    # Just look at goods consumed in quantities greater than error tolerance
    Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms',
                                                          title='Effects of Price Changes on Subsistence Diet Composition Based on Egg Price for '+group)

/tmp/ipykernel_198/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



## Total Cost for Population of Interest

Our population of interest is all UC berkeley students and we assume they are males and females from 19-30. Based on data from UC Berkeley Office of Planning and Analysis(https://opa.berkeley.edu/campus-data/uc-berkeley-quick-facts), there are 23,974 female students and 20,642 male students in Berkeley for Fall 2022. 

In [23]:
#Cost per capita for female and male student
cost_f_1930 = cost_tbl.loc[:,'F 19-30'].min()
cost_m_1930 = cost_tbl.loc[:,'M 19-30'].min()

num_f = 23974
num_m = 20642

#Calculation of total cost for all students
total_cost = cost_f_1930*num_f + cost_m_1930*num_m
total_cost

240099.02962844836

### Total Food Required for Population of Interest

Still, we consider to feed the population by Ttoust, our cheapest option. 

In [24]:
#Extract result for male 19-30 in Ttoust
group = 'M 19-30'
tol = 1e-5

result = solve_subsistence_problem(ing_res[1],ing_prices[1],dri_min[group],dri_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

Cost of diet for M 19-30 is $5.41 per day.


Diet (in 100s of grams or milliliters):
Egg                0.748143
Purple Cabbage     0.515229
Spinach            6.186931
Pork               2.443126
Flour              1.978106
dtype: float64



/tmp/ipykernel_198/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



In [25]:
#Extract result for female 19-30 in Ttoust
group = 'F 19-30'
tol = 1e-5

result = solve_subsistence_problem(ing_res[1],ing_prices[1],dri_min[group],dri_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

Cost of diet for F 19-30 is $5.35 per day.


Diet (in 100s of grams or milliliters):
Egg                0.899094
Purple Cabbage     0.256304
Spinach            6.162313
Pork               2.213320
Flour              2.310570
dtype: float64



/tmp/ipykernel_198/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



In [26]:
#Food requirement
#Male
egg_m = 0.748143
pc_m = 0.515229
spinach_m = 6.186931
pork_m = 2.443126
flour_m = 1.978106

#Female
egg_f = 0.899094
pc_f = 0.256304
spinach_f = 6.162313
pork_f = 2.213320
flour_f = 2.310570

#Total
egg = egg_m*num_m + egg_f*num_f
pc = pc_m*num_m + pc_f*num_f
spinach = spinach_m*num_m + spinach_f*num_f
pork = pork_m*num_m + pork_f*num_f
flour = flour_m*num_m + flour_f*num_f

In [27]:
print('We need', egg/10, 'kg eggs to feed all UC Berkeley students for one day')
print('We need', pc/10, 'kg purple cabbage to feed all UC Berkeley students for one day')
print('We need', spinach/10, 'kg spinach to feed all UC Berkeley students for one day')
print('We need', pork/10, 'kg pork to feed all UC Berkeley students for one day')
print('We need', flour/10, 'kg of flour to feed all UC Berkeley students for one day')

We need 3699.8047361999998 kg eggs to feed all UC Berkeley students for one day
We need 1677.9989114 kg purple cabbage to feed all UC Berkeley students for one day
We need 27544.592156400002 kg spinach to feed all UC Berkeley students for one day
We need 10349.314057200001 kg pork to feed all UC Berkeley students for one day
We need 9622.566923199998 kg of flour to feed all UC Berkeley students for one day
